In [153]:
import geopandas as gpd
from geopandas import GeoDataFrame

import pandas as pd
from shapely.geometry import Point,LineString
import networkx as nx
from shapely.ops import split, snap
from shapely.geometry import Point, LineString
import numpy as np

In [44]:
# Read the road dataset
roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco.gpkg")
# Dissolve the road lines to merge connected segments
dissolved_roads = roads.dissolve(by='LINEARID') 


# Intersections & Deadends

In [ ]:
# Extract the start and end points of each line
start_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[0]))
end_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[-1]))
# Concatenate the start and end points into a single GeoSeries using pd.concat()
all_points = gpd.GeoSeries(pd.concat([start_points, end_points]), crs=roads.crs)
# Count the occurrence of each point
point_counts = all_points.value_counts()


# Filter the points that appear only once (intersections & deadends)
intersections_deadends = point_counts[point_counts == 1]
# Create a dictionary with the dead end points and their counts
intersections_deadends_data = {'geometry': intersections_deadends.index, 'count': intersections_deadends.values}
 #Create a GeoDataFrame with the dead end points
intersections_deadends_data_points = gpd.GeoDataFrame(intersections_deadends_data, crs=roads.crs)
# Save the dead end points to a new shapefile
intersections_deadends_data_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_intersection_deadends.gpkg", driver='GPKG')


# Deadends
## converts a dataset of roads into a graph representation and identifies dead-end points within that network.

* Graph Initialization: The process begins with the initialization of a graph. In this context, a graph is a mathematical structure used to model pairwise relations between objects. Here, roads and intersections are modeled as edges and nodes, respectively.
* Geometry Simplification and Conversion: Each road segment (line geometry) from the dataset is simplified to remove minor irregularities and potential complexities. This simplification helps in avoiding inaccuracies when converting these geometries into a graph format. The simplified line geometries are then converted into a series of points (nodes in the graph), and consecutive points are connected by edges. This effectively builds the network where paths (edges) between points (nodes) represent segments of the road.
* Identification of Dead Ends: Once the graph is built, the code identifies dead ends in the network. A dead end is recognized by having a node with only one connected edge (degree of 1). This means there is no alternate path from that node, signifying the end of a road without any junctions.

In [ ]:
roads_dissolved = roads.dissolve(by='LINEARID') 
# Initialize a new graph. This will be used to store the road network where nodes represent junctions or endpoints of roads, and edges represent the roads themselves.
G = nx.Graph()

# Iterate over each row in the dissolved roads GeoDataFrame. Each row corresponds to a road segment.
for index, row in roads_dissolved.iterrows():
    # Simplify the geometry of the road segment to remove minor variations and potential self-intersections. This is important for accurately converting the geometries into a graph structure without unnecessary complexity.
    line = row.geometry.simplify(tolerance=0.01)
    
    # Check if the simplified geometry is a LineString (a simple line shape). We only want to process line shapes as these represent the roads.
    if isinstance(line, LineString):
        # Convert the line's coordinates into a list of tuples. Each tuple represents a point (node) on the line.
        points = list(map(tuple, line.coords))
        
        # Add these points as a path in the graph. This connects each consecutive pair of points with an edge, effectively building the road network in graph form.
        nx.add_path(G, points)

# Identify all nodes in the graph that have a degree of 1. A node with a degree of 1 only has one connection (edge) to another node, which means it is an endpoint without any further continuation - a dead end.
dead_ends = [node for node, degree in G.degree() if degree == 1]

# Convert the list of dead end nodes (points) back into a GeoDataFrame for further spatial analysis. This allows us to use geographic functions on the points.
dead_ends_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([p[0] for p in dead_ends], [p[1] for p in dead_ends]), crs=roads.crs)


In [ ]:
# Save the dead end points to a new shapefile
dead_ends_gdf.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg", driver='GPKG')

# Intersections

In [ ]:
# Perform a spatial difference
 #This operation returns the geometric difference of points in intersections_deadends that are not in dead_end_points.
true_intersections = gpd.overlay(intersections_deadends_data_points, dead_ends_gdf, how='difference')

# Now true_intersections contains only the intersection points that are not dead ends.
# Optionally, save to file
true_intersections.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_intersections.gpkg", driver='GPKG')

# QGIS
* Using roads layer use ***Split with line*** using roads twice
* using  '***Extract specific vertices***' with 0, -1 as input on 'Split' layer, produces 'Vertices'
* Take difference between 'Vertices' and 'sbco_deadends' produced from this notebook, produces 'Difference'
* Using 'Difference' again take difference using 'true_intersection' produced from this notebook, this produces another 'Differece' layer which is the remaining intersections not captured in this code.
* edit attribute table with field calculator, existing column 'fid', @row_number
Result is dependent on quality of original road layer. REQUIRES manual inspection, especially in long isolated roads and in urban areas with many intersections. In long roads a random vertix can be sometimes found that ties one road together. In urban areas with high number of intersections the many vertices are harder to capture. 

If 100% coverage of intersections is desired, — this analysis doesn't absolutely require it as the roads that are missing intersections reach an intersection or deadend — then the reccomendation is that after manual inspection and removal of unwanted points, merge the three layers of sbco_deadends, sbco_intersections, and the final difference layer. In qgis use the extract all vertices tool and take the difference with the merged layer. This will produce an extreme amount of points, create a blank shapefile, cut and paste the intersection points desired manually.

Using simplify on roads  may help reduce when extracting all vertices.

In [85]:
missing_vertices = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_missing_vertices.gpkg")

In [86]:
# Check for NaN coordinates in each geometry
valid_points = missing_vertices[~missing_vertices.geometry.apply(lambda geom: np.isnan(geom.x) or np.isnan(geom.y))]


In [88]:
valid_points['type'] = 'intersection'

c:\Users\bsf31\.conda\envs\dev_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [89]:
valid_points

,LINEARID,FULLNAME,RTTYP,MTFCC,vertex_pos,vertex_index,vertex_part,vertex_part_index,distance,angle,geometry,type
1,110809347721,Cravens Ln Exd,M,S1400,-1,1,0,1,78.233719,10.420668,POINT (6097243.786 1973719.509),intersection
5,1102543203587,Pt Sal Lp,M,S1400,-1,14,0,14,274.666627,110.052047,POINT (5815149.026 2069884.831),intersection
6,1102543203587,Pt Sal Lp,M,S1400,0,0,0,0,0.000000,110.309265,POINT (5815149.026 2069884.831),intersection
7,1102543203587,Pt Sal Lp,M,S1400,-1,9,0,9,241.007612,100.580528,POINT (5815378.910 2069813.990),intersection
8,1102543203587,Pt Sal Lp,M,S1400,0,0,0,0,0.000000,100.064854,POINT (5815378.910 2069813.990),intersection
...,...,...,...,...,...,...,...,...,...,...,...,...
71862,1103678161715,Refugio Rd,M,S1400,0,0,0,0,0.000000,0.978029,POINT (5930519.240 2055033.225),intersection
71863,1103678161715,Refugio Rd,M,S1400,-1,1,0,1,791.184043,0.978029,POINT (5930532.744 2055824.294),intersection
71864,110809630261,NaN,NaN,S1400,0,0,0,0,0.000000,273.363971,POINT (5928444.462 2048626.407),intersection
71865,110809630261,NaN,NaN,S1400,-1,5,0,5,259.559456,308.284647,POINT (5928189.813 2048654.309),intersection


In [90]:
valid_points = valid_points[['type', 'geometry']]

In [91]:
valid_points

,type,geometry
1,intersection,POINT (6097243.786 1973719.509)
5,intersection,POINT (5815149.026 2069884.831)
6,intersection,POINT (5815149.026 2069884.831)
7,intersection,POINT (5815378.910 2069813.990)
8,intersection,POINT (5815378.910 2069813.990)
...,...,...
71862,intersection,POINT (5930519.240 2055033.225)
71863,intersection,POINT (5930532.744 2055824.294)
71864,intersection,POINT (5928444.462 2048626.407)
71865,intersection,POINT (5928189.813 2048654.309)


In [92]:
valid_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\valid_vertices.gpkg", driver='GPKG')

In [93]:

filepath = r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg"
sbco_deadends = gpd.read_file(filepath)


filepath = r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_intersections.gpkg"
sbco_intersections = gpd.read_file(filepath)


In [94]:
sbco_deadends['type'] = 'deadend'
sbco_deadends = sbco_deadends[['type', 'geometry']]
sbco_deadends

,type,geometry
0,deadend,POINT (5904160.024 2053103.671)
1,deadend,POINT (5903736.236 2052907.586)
2,deadend,POINT (5903977.278 2052796.051)
3,deadend,POINT (5843767.662 2175017.699)
4,deadend,POINT (6014387.718 1987284.142)
...,...,...
6277,deadend,POINT (5798963.779 2106449.489)
6278,deadend,POINT (5801624.549 2104342.964)
6279,deadend,POINT (5801429.841 2104044.978)
6280,deadend,POINT (6016530.887 1989747.508)


In [95]:
sbco_intersections['type'] = 'intersection'
sbco_intersections = sbco_intersections[['type', 'geometry']]
sbco_intersections

,type,geometry
0,intersection,POINT (5799291.766 2105909.325)
1,intersection,POINT (5801479.067 2105701.256)
2,intersection,POINT (6021656.895 1985006.616)
3,intersection,POINT (5826447.443 2069820.069)
4,intersection,POINT (5830342.307 2149864.353)
...,...,...
9882,intersection,POINT (5819887.064 2071555.588)
9883,intersection,POINT (6043526.431 1990361.400)
9884,intersection,POINT (5822012.393 2071893.739)
9885,intersection,POINT (5787570.605 2182073.684)


In [96]:
deadends_intersections = gpd.GeoDataFrame(pd.concat([sbco_deadends, sbco_intersections, valid_points], ignore_index=True))


In [97]:
deadends_intersections.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\clean_deadends_intersections.gpkg", driver='GPKG')

In [98]:
rwmp = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\RWMP_AREA_2229.gpkg")

In [99]:
rwmp_deadends_intersections = gpd.clip(deadends_intersections, rwmp)

In [100]:
rwmp_deadends_intersections.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\rwmp_deadends_intersections.gpkg", driver='GPKG')

# Single Egress Points

In [117]:
rmwp_road = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\rwmp_road_split.gpkg")


In [124]:
roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco_split.gpkg")


In [120]:
# Filter points to get intersections and deadends
intersections = deadends_intersections[deadends_intersections['type'] == 'intersection']
deadends = deadends_intersections[deadends_intersections['type'] == 'deadend']





In [132]:
# Initialize list to hold single egress roads
single_egress_roads = []

# Loop through each road in the dataset
for index, road in roads.iterrows():
    # Check if the road intersects with dead end points
    intersects_deadends = deadends.intersects(road.geometry)
    if sum(intersects_deadends) == 1:  # Only one dead end
        single_egress_roads.append(road)

# Convert list to GeoDataFrame
single_egress_roads = gpd.GeoDataFrame(single_egress_roads, columns=roads.columns, crs=roads.crs)



In [133]:
single_egress_roads

,LINEARID,FULLNAME,RTTYP,MTFCC,geometry
1,110809346329,Orchid Dr Exd,M,S1400,"MULTILINESTRING ((6017612.669 1980535.398, 601..."
2,1102543203587,Pt Sal Lp,M,S1400,"MULTILINESTRING ((5814880.387 2069933.333, 581..."
14,1102529073203,Old Tisbury Ln,M,S1400,"MULTILINESTRING ((5840499.972 2142413.666, 584..."
21,110809364444,Old Santa Rosa Rd,M,S1740,"MULTILINESTRING ((5927471.259 2040222.243, 592..."
77,110809363430,Old Tepusquet Rd,M,S1400,"MULTILINESTRING ((5893444.358 2173154.373, 589..."
...,...,...,...,...,...
35922,110809636441,NaN,NaN,S1740,"MULTILINESTRING ((5933545.133 2078372.256, 593..."
35925,110809637060,NaN,NaN,S1740,"MULTILINESTRING ((5923076.812 2099784.223, 592..."
35927,110809631927,NaN,NaN,S1400,"MULTILINESTRING ((5885011.432 2118606.538, 588..."
35928,110809364995,Dove Meadow Ln,M,S1400,"MULTILINESTRING ((5927641.490 2054929.633, 592..."


In [127]:
single_egress_roads.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\single_egress_roads_sbco.gpkg", driver='GPKG')

In [159]:
#  Identify intersection points for single egress roads
def find_intersections(egress_roads, all_intersections):
    # Properly initialize an empty GeoDataFrame with a geometry column
    egress_points = gpd.GeoDataFrame(columns=['geometry'], crs=egress_roads.crs)
    
    for road in egress_roads.geometry:
        # Buffer the road slightly to ensure all potential intersections are captured
        road_buffer = road.buffer(5)  # Adjust the buffer size as needed based on your spatial unit
        # Find intersections that are within the road buffer
        intersecting_points = all_intersections[all_intersections.intersects(road_buffer)]
        # Concatenate current egress_points with new intersecting_points
        egress_points = pd.concat([egress_points, intersecting_points], ignore_index=True)

    # Ensure the result is a GeoDataFrame with the correct CRS
    egress_points = gpd.GeoDataFrame(egress_points, geometry='geometry', crs=egress_roads.crs)
    
    return egress_points.drop_duplicates(subset='geometry')



In [160]:
egress_points = find_intersections(single_egress_roads, intersections)

In [163]:
egress_points = egress_points[['type', 'geometry']]
egress_points

,type,geometry
0,intersection,POINT (6017612.669 1980535.398)
1,intersection,POINT (5815149.026 2069884.831)
5,intersection,POINT (5840409.979 2142277.117)
6,intersection,POINT (5929303.108 2041439.711)
7,intersection,POINT (5893444.358 2173154.373)
...,...,...
14122,intersection,POINT (5919457.781 2061339.303)
14125,intersection,POINT (5934045.779 2078575.160)
14128,intersection,POINT (5924499.975 2103575.949)
14130,intersection,POINT (5885011.432 2118606.538)


In [164]:
egress_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\egress_points_sbco.gpkg", driver='GPKG')

In [74]:
# Convert intersection points to a MultiPoint object (necessary for the split operation)
intersection_points = intersections.geometry.unary_union


In [76]:
def split_roads(road, points):
    if points.intersects(road.geometry):
        return split(road.geometry, points)
    else:
        return [road.geometry]

# Apply the split function to each road segment
split_roads_layer = rmwp_road.copy()
split_roads_layer['geometry'] = split_roads_layer.apply(lambda x: split_roads(x, intersection_points), axis=1)


c:\Users\bsf31\.conda\envs\dev_env\Lib\site-packages\shapely\linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\bsf31\AppData\Local\Temp\ipykernel_4284\3688002383.py:9: UserWarning: Geometry column does not contain geometry.
  split_roads_layer['geometry'] = split_roads_layer.apply(lambda x: split_roads(x, intersection_points), axis=1)


In [77]:
# Since split can create multiple geometries, you need to 'explode' them into separate rows
split_roads_layer = split_roads_layer.explode().reset_index(drop=True)


In [83]:
split_roads_layer = gpd.GeoDataFrame(split_roads_layer, crs=rmwp_road.crs)


In [84]:
split_roads_layer.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\split_roads_layer.gpkg", driver='GPKG')

In [ ]:
# # Identify nodes that, when removed, increase the number of connected components - these nodes represent single egress points.
# cut_points = list(nx.articulation_points(G))

# # Convert these nodes back to a GeoDataFrame for spatial analysis
# cut_points_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([p[0] for p in cut_points], [p[1] for p in cut_points]))

In [ ]:
# # Assuming roads_dissolved is your GeoDataFrame of roads
# G = nx.Graph()

# # Convert line geometries to graph, focusing only on true endpoints
# for line in roads_dissolved.geometry:
#     if isinstance(line, LineString):
#         # Simplify the line to minimize unnecessary intermediate points
#         simplified_line = line.simplify(tolerance=0.01)
#         endpoints = [simplified_line.coords[0], simplified_line.coords[-1]]
#         nx.add_path(G, endpoints)

# # Use DBSCAN or similar to cluster endpoints to mitigate multiple close points issue
# coords = [point for point, degree in G.nodes(data=True) if G.degree(point) == 1]
# db = DBSCAN(eps=10, min_samples=1).fit(coords)  # Adjust eps based on your spatial resolution needs
# clusters = db.labels_

# # Convert clustered endpoints back to a GeoDataFrame
# clustered_points = [MultiPoint([coords[i] for i in range(len(coords)) if clusters[i] == k]).centroid for k in set(clusters)]
# dead_ends_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([p.x for p in clustered_points], [p.y for p in clustered_points]))


In [ ]:
# Identifying articulation points and analyzing their impact
articulation_points = list(nx.articulation_points(G))
impact_dict = {}

# Analyzing the size of components when an articulation point is removed
for point in articulation_points:
    G_temp = G.copy()
    G_temp.remove_node(point)
    components = list(nx.connected_components(G_temp))
    # Find the largest component size when the point is removed
    largest_component_size = max([len(comp) for comp in components])
    total_size = len(G_temp.nodes())
    isolated_size = total_size - largest_component_size
    # Store the impact information
    impact_dict[point] = isolated_size

# Filtering to find significant single egress points
significant_points = {k: v for k, v in impact_dict.items() if v > 50}  # adjust 50 to your specific threshold

# Convert significant articulation points back to GeoDataFrame
significant_points_gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy([p[0] for p in significant_points.keys()], [p[1] for p in significant_points.keys()]),
    crs=roads.crs
)


In [ ]:
# Save the dead end points to a new shapefile
significant_points_gdf.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_single_egress_points2.gpkg", driver='GPKG')

In [ ]:
# # Extract the start and end points of each line
# start_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[0]))
# end_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[-1]))
# # Concatenate the start and end points into a single GeoSeries using pd.concat()
# all_points = gpd.GeoSeries(pd.concat([start_points, end_points]), crs=roads.crs)
# # Count the occurrence of each point
# point_counts = all_points.value_counts()


# # Filter the points that appear only once (dead ends)
# dead_ends = point_counts[point_counts == 1]
# # Create a dictionary with the dead end points and their counts
# dead_end_data = {'geometry': dead_ends.index, 'count': dead_ends.values}
#  #Create a GeoDataFrame with the dead end points
# dead_end_points = gpd.GeoDataFrame(dead_end_data, crs=roads.crs)
# # Save the dead end points to a new shapefile
# dead_end_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg", driver='GPKG')
# #2

# # Create a spatial index for the road lines
# spatial_index = dissolved_roads.sindex

# # Function to check if a point is a dead end
# def is_dead_end(point):
#     # Create a small buffer around the point
#     buffer_dist = 1e-5  # Adjust the buffer distance as needed
#     buffer = point.buffer(buffer_dist)
    
#     # Find the indices of the lines that intersect the buffer
#     possible_matches_index = list(spatial_index.intersection(buffer.bounds))
#     possible_matches = dissolved_roads.iloc[possible_matches_index]
    
#     # Check if the point touches exactly one line geometry
#     touching_lines = [line for line in possible_matches.geometry if point.touches(line)]
    
#     return len(touching_lines) == 1
# # Function to check if a point is a dead end
# def is_dead_end(point):
#     # Create a small buffer around the point
#     buffer_dist = 1e-8  # Adjust the buffer distance as needed
#     buffer = point.buffer(buffer_dist)
    
#     # Find the indices of the lines that intersect the buffer
#     possible_matches_index = list(spatial_index.intersection(buffer.bounds))
#     possible_matches = dissolved_roads.iloc[possible_matches_index]
    
#     # Check if the point touches any line geometries
#     touches_line = any(point.touches(line) for line in possible_matches.geometry)
    
#     return not touches_line
 

# # Identify the dead end points
# dead_ends2 = all_points[all_points.apply(is_dead_end)]
# # Create a GeoDataFrame with the dead end points
# dead_end_points2 = gpd.GeoDataFrame(geometry=dead_ends2, crs=roads.crs)


# # Read the road dataset
# roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco.gpkg")
# # Dissolve the road lines to merge connected segments
# dissolved_roads = roads.dissolve(by='LINEARID') 
# # Reset the index of dissolved_roads
# dissolved_roads = dissolved_roads.reset_index()
# # Extract the start and end points of each line
# start_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[0]))
# end_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[-1]))
# # Concatenate the start and end points into a single GeoSeries
# all_points = pd.concat([start_points, end_points])

# # Create a DataFrame where LINEARID is repeated for each start and end point
# repeated_linearid = dissolved_roads.loc[dissolved_roads.index.repeat(2), 'LINEARID'].reset_index(drop=True)
# # Create a GeoDataFrame with all points and their corresponding LINEARID
# all_points_gdf = gpd.GeoDataFrame({'geometry': all_points, 'LINEARID': repeated_linearid})

# # Count the number of occurrences of each LINEARID for each point
# linearid_counts = all_points_gdf.groupby(all_points_gdf.geometry)['LINEARID'].count()

# # Identify the dead end points (points with only one occurrence of LINEARID)
# dead_ends = linearid_counts[linearid_counts == 1].index

# # Create a GeoDataFrame with the dead end points
# dead_end_points = gpd.GeoDataFrame(geometry=dead_ends, crs=roads.crs)

# # Save the dead end points to a new shapefile
# dead_end_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg", driver='GPKG')
# # Load the road data
# roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco.gpkg")

